In [1]:
import csv
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon

In [2]:
data = pd.read_csv("project3/data/joined_raw_airports.csv")

In [3]:
map_df = data.drop(columns=["id","carrier_ct", "carrier", "carrier_name", "weather_ct", "nas_ct","security_ct","late_aircraft_ct", "arr_diverted", "arr_delay","carrier_delay", "weather_delay","nas_delay", "security_delay", "late_aircraft_delay", "id-2","country_code"])
map_df.head()

,year,month,airport_name,arr_flights,arr_del15,arr_cancelled,iata,airport,latitude,longitude
0,2020,12,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",5985,445,5,ATL,Hartsfield-Jackson Atlanta International Airport,33.6367,-84.4281
1,2020,12,"Boston, MA: Logan International",21,3,0,BOS,Logan International Airport,42.3643,-71.0052
2,2020,12,"Baltimore, MD: Baltimore/Washington Internatio...",96,17,1,BWI,Baltimore/Washington International Airport,39.1753,-76.6683
3,2020,12,"Charlotte, NC: Charlotte Douglas International",42,1,0,CLT,Charlotte Douglas International Airport,35.2140,-80.9431
4,2020,12,"Washington, DC: Ronald Reagan Washington National",37,1,1,DCA,Ronald Reagan Washington National Airport,38.8522,-77.0378


In [4]:
airports_table = pd.read_csv("airport_table.csv")
display(airports_table)

,Unnamed: 0,iata,airport,latitude,longitude
0,6671,SLC,Salt Lake City International Airport,40.7884,-111.9780
1,7019,ATL,Hartsfield-Jackson Atlanta International Airport,33.6367,-84.4281
2,7102,FLL,Fort Lauderdale-Hollywood International Airport,26.0726,-80.1527
3,7103,MIA,Miami International Airport,25.7932,-80.2906
4,7151,MCO,Orlando International Airport,28.4294,-81.3090
5,7337,MSP,Minneapolis-Saint Paul International Airport (...,44.8820,-93.2218
6,7651,LAX,Los Angeles International Airport,33.9425,-118.4080
7,7656,LAS,Harry Reid International Airport,36.0801,-115.1520
8,7800,DFW,Dallas/Fort Worth International Airport,32.8968,-97.0380
9,7805,DEN,Denver International Airport,39.8617,-104.6730


In [5]:
m_airports = folium.Map(location=(39.8283, -98.5795),tiles="Cartodb dark_matter", zoom_start=4)

In [6]:
# calculate totals per airport
arr_airports = map_df.groupby('iata')["arr_flights"].sum()
delay_airports = map_df.groupby('iata')['arr_del15'].sum()
cancel_airports = map_df.groupby('iata')['arr_cancelled'].sum()

#calculate percentages
delay_percents = round((delay_airports/arr_airports)*100, 3)
cancel_percents = round((cancel_airports/arr_airports)*100, 3)

#gather latitude and longitue
airport_lat = map_df.groupby('iata')['latitude'].unique()
airport_lng = map_df.groupby('iata')['longitude'].unique()
airport_iata = map_df.groupby('iata')['airport'].unique()

In [7]:
# Convert the DataFrame columns to native Python lists and types
lat = airport_lat.astype(float).tolist()
lng = airport_lng.astype(float).tolist()

delay = delay_percents.astype(float).tolist()
cancel = cancel_percents.astype(float).tolist()

# Zip the lists for folium format
delay_heat = list(zip(lat, lng, delay))
cancel_heat = list(zip(lat, lng, cancel))

In [8]:
# Create map features

marker_cluster = MarkerCluster(name="Airports").add_to(m_airports)

for row in airports_table.itertuples():
    folium.Marker(
        location=[row.latitude, row.longitude],
        popup=(f"{row.iata}: {row.airport}"),
        icon=folium.Icon(color="blue", icon="plane")
    ).add_to(marker_cluster)

# add delays to layer control
fg1 = folium.FeatureGroup(name="Delays", control=True).add_to(m_airports)
HeatMap(delay_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg1)

# add cancellations to layer control
fg2 = folium.FeatureGroup(name="Cancellations", control=True).add_to(m_airports)
HeatMap(cancel_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg2)

# add layer control to the map
folium.LayerControl(collapsed=False).add_to(m_airports)
m_airports